In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

from isic_helper import compute_pauc, compute_auc

In [2]:
# model_names = ["cb", "lgb", "resnet18"]
# versions = ["v1", "v1", "v1"]
# paths = ["/kaggle/input/isic-scd-cb-train", "/kaggle/input/isic-scd-lgb-train", "/kaggle/input/isic-scd-resnet18-train"]
# target_column = "final_target"
# wts = [0.35, 0.35, 0.3]

# model_names = ["cb", "lgb"]
# versions = ["v1", "v1", "v1"]
# paths = ["/kaggle/input/isic-scd-cb-train", "/kaggle/input/isic-scd-lgb-train"]
# target_column = "final_target"
# wts = [0.5, 0.5]

model_names = ["lgb", "resnet18"]
versions = ["v1", "v1"]
paths = ["/kaggle/input/isic-scd-lgb-train", "/kaggle/input/isic-scd-resnet18-train"]
target_column = "final_target"
wts = [0.65, 0.35]

In [3]:
subs = {}
cv_pauc_oof_avg = {}
for i in range(len(model_names)):
    model_name = model_names[i]
    version = versions[i]
    path = paths[i]
    subs[model_name] = pd.read_csv(f"{path}/oof_preds_{model_name}_{version}.csv")
    cv_pauc_oof_avg[model_name] = {}
cv_pauc_oof_avg["ensemble"] = {}

folds = np.sort(subs[model_names[0]]["fold"].unique())
for fold in folds:
    fold_ens_preds = []
    for idx, (version, (model_name, df)) in enumerate(zip(versions, subs.items())):
        fold_df = df[df["fold"] == fold]
        fold_model_preds = fold_df[f"oof_{model_name}_{version}"]
        cv_pauc_oof_avg[model_name][f"fold_{fold}"] = compute_pauc(fold_df[target_column], fold_model_preds)
        fold_ens_preds.append(wts[idx] * fold_df[f"oof_{model_name}_{version}"].rank(pct=True).values)
    cv_pauc_oof_avg["ensemble"][f"fold_{fold}"] = compute_pauc(fold_df[target_column], np.sum(fold_ens_preds, axis=0))

In [4]:
for model_name in model_names:
    print(f"{model_name}:\nFold scores: {cv_pauc_oof_avg[model_name]}\n Avg score: {np.mean(list(cv_pauc_oof_avg[model_name].values()))}")
print(f"Ensemble:\nFold scores: {cv_pauc_oof_avg['ensemble']}\n Avg score: {np.mean(list(cv_pauc_oof_avg['ensemble'].values()))}")

lgb:
Fold scores: {'fold_1': 0.16189363377553598, 'fold_2': 0.16460742359112585, 'fold_3': 0.1566555858531654, 'fold_4': 0.17275461431905254, 'fold_5': 0.16268287305687626}
 Avg score: 0.16371882611915117
resnet18:
Fold scores: {'fold_1': 0.15586849727633634, 'fold_2': 0.15323922173948376, 'fold_3': 0.14612993098800708, 'fold_4': 0.15783404674844936, 'fold_5': 0.15046869195347348}
 Avg score: 0.15270807774115
Ensemble:
Fold scores: {'fold_1': 0.16760549182900902, 'fold_2': 0.17149450951707168, 'fold_3': 0.16299309880071378, 'fold_4': 0.17566750071757492, 'fold_5': 0.16728517365520842}
 Avg score: 0.1690091549039156
